In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import Session
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from tqdm import tqdm
import warnings

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# sess = Session(config=config)

warnings.filterwarnings("ignore")

c:\users\metroide\pycharmprojects\speech\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\metroide\pycharmprojects\speech\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\metroide\pycharmprojects\speech\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\metroide\pycharmprojects\speech\ve

In [2]:
train_audio_path = './train/audio/'
all_wave = []
all_label = []

In [5]:
# Subsample
labels = [
    'left', 'go', 'yes', 'down', 'up', 'on', 'right', 'no', 'off', 'stop',
]

In [19]:

train_audio_path = './datas/train/audio'

all_wave = []
all_label = []
for label in tqdm(labels):
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        samples, sample_rate = librosa.load(train_audio_path + '/' + label + '/' + wav, sr = 16000)
        samples = librosa.resample(samples, sample_rate, 8000)
        if(len(samples)== 8000) : 
            all_wave.append(samples)
            all_label.append(label)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [12:39<00:00, 75.93s/it]


In [20]:
#remove this after oploading real dataset
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
label_enconder = LabelEncoder()
y = label_enconder.fit_transform(all_label)
classes = list(label_enconder.classes_)
y = np_utils.to_categorical(y, num_classes=len(labels))
print(y)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [21]:
#reshape datas
all_wave = np.array(all_wave).reshape(-1,8000,1)

In [22]:
#create train and validation set 20% pour le text et 80% pour l'entrainement
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(np.array(all_wave),np.array(y),stratify=y,test_size = 0.2,random_state=777,shuffle=True)

In [26]:
from keras.layers import Bidirectional, BatchNormalization, CuDNNGRU, TimeDistributed, GRU
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
K.clear_session()

In [27]:
#definition du model en utilisaant keras
inputs = Input(shape=(8000,1))
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(inputs)

#First Conv1D layer
x = Conv1D(8,13, padding='valid', activation='relu', strides=1)(x)
x = MaxPooling1D(3)(x)
x = Dropout(0.3)(x)

#Second Conv1D layer
x = Conv1D(16, 11, padding='valid', activation='relu', strides=1)(x)
x = MaxPooling1D(3)(x)
x = Dropout(0.3)(x)

#Third Conv1D layer
x = Conv1D(32, 9, padding='valid', activation='relu', strides=1)(x)
x = MaxPooling1D(3)(x)
x = Dropout(0.3)(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(x)
x = Bidirectional(GRU(128, return_sequences=True), merge_mode='sum')(x)
x = Bidirectional(GRU(128, return_sequences=True), merge_mode='sum')(x)
x = Bidirectional(GRU(128, return_sequences=False), merge_mode='sum')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True)(x)

#Flatten layer
# x = Flatten()(x)
#Dense Layer 1
x = Dense(256, activation='relu')(x)
outputs = Dense(len(labels), activation="softmax")(x)
model = Model(inputs, outputs)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8000, 1)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8000, 1)           4         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 7988, 8)           112       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2662, 8)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2662, 8)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2652, 16)          1424      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 884, 16)           0         
__________

In [28]:
#compilation du model en utilisant la fonction d'erreur caegorical cross entropie
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
#definition des differents callback pour l'arret du model et sauvegarde du meilleur model
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15, min_delta=0.0001) 
checkpoint = ModelCheckpoint('speech2text_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [ ]:
#entrainement du model avec un batch size de 32 sur le CPU.
hist = model.fit(
    x=x_train, 
    y=y_train,
    epochs=100, 
    callbacks=[early_stop, checkpoint], 
    batch_size=32, 
    validation_data=(x_valid,y_valid)
)

Instructions for updating:
Use tf.cast instead.
Train on 17049 samples, validate on 4263 samples
Epoch 1/100
17049/17049 [==============================] - 1359s 80ms/step - loss: 1.2798 - acc: 0.5397 - val_loss: 0.9819 - val_acc: 0.6716

Epoch 00001: val_acc improved from -inf to 0.67159, saving model to speech2text_model.hdf5
Epoch 2/100
17049/17049 [==============================] - 1481s 87ms/step - loss: 0.6868 - acc: 0.7647 - val_loss: 0.6834 - val_acc: 0.7795

Epoch 00002: val_acc improved from 0.67159 to 0.77950, saving model to speech2text_model.hdf5
Epoch 3/100
17049/17049 [==============================] - 1433s 84ms/step - loss: 0.5310 - acc: 0.8165 - val_loss: 0.5535 - val_acc: 0.8198

Epoch 00003: val_acc improved from 0.77950 to 0.81985, saving model to speech2text_model.hdf5
Epoch 4/100
17049/17049 [==============================] - 1566s 92ms/step - loss: 0.4686 - acc: 0.8429 - val_loss: 0.3943 - val_acc: 0.8670

Epoch 00004: val_acc improved from 0.81985 to 0.86700, s